In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

In [ ]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir

# activation: relu, sigmoid, tanh
# weight initializer: he, xavier, normal, truncated_normal
# optimizer: Adam, Momentum, Gradient Descent.

In [ ]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

In [ ]:
x = tf.placeholder(tf.float32, [None, 3072])

# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# vgg-net思想
def convnet(inputs, activation, kernel_initializer):
    conv1_1 = tf.layers.conv2d(inputs,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            activation=activation,
                            kernel_initializer = kernel_initializer, # default: tf.glorot_uniform_initializer
                            name='conv1_1')

    conv1_2 = tf.layers.conv2d(conv1_1,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            kernel_initializer = kernel_initializer,    
                            activation=activation,
                            name='conv1_2')

    # 16 * 16
    pooling1 = tf.layers.max_pooling2d(conv1_2,
                                      (2,2), # kernel size
                                      (2,2), # stride
                                      name='pool1')

    conv2_1 = tf.layers.conv2d(pooling1,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            kernel_initializer = kernel_initializer, 
                            activation=activation,
                            name='conv2_1')

    conv2_2 = tf.layers.conv2d(conv2_1,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            kernel_initializer = kernel_initializer, 
                            activation=activation,
                            name='conv2_2')

    # 8 * 8
    pooling2 = tf.layers.max_pooling2d(conv2_2,
                                      (2,2), # kernel size
                                      (2,2), # stride
                                      name='pool2')

    conv3_1 = tf.layers.conv2d(pooling2,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            kernel_initializer = kernel_initializer, 
                            activation=activation,
                            name='conv3_1')

    conv3_2 = tf.layers.conv2d(conv3_1,
                            32, # output channel number
                            (3,3), # kernel size
                            padding = 'same', 
                            kernel_initializer = kernel_initializer, 
                            activation=activation,
                            name='conv3_2')
    # 4 * 4 * 32
    pooling3 = tf.layers.max_pooling2d(conv3_2,
                                      (2,2), # kernel size
                                      (2,2), # stride
                                      name='pool3')

    # [None, 4 * 4 * 32]
    flatten = tf.layers.flatten(pooling3)
    return flatten
    
# flatten = convnet(x_image, tf.nn.relu) # train 10k: 73.35%
# flatten = convnet(x_image, tf.nn.relu, None)  # train 76.53% 100k train
# flatten = convnet(x_image, tf.nn.relu, tf.truncated_normal_initializer(stddev=0.02))  # 78.04% 100k train
# flatten = convnet(x_image, tf.nn.relu, tf.keras.initializers.he_normal)  # 71.52% 100k train

flatten = convnet(x_image, tf.nn.sigmoid, None) # train 10k: 53.39%

y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
#     train_op = tf.train.GredientDescentOptimizer(1e-4).minimize(loss) # 12.35% train 100k
#     train_op = tf.train.MomentumOptimizer(learning_rate=1e-4, momentum).minimize(loss) #35.75% train 100k
#   reason: 1.initializer incorrect. 2. 不充分的训练
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps =100

output_summary_every_steps = 100

with tf.Session() as sess:
    sess.run(init)
    # 2. 训练过程中将这些变量计算出来，输出到文件中
    
    fixed_test_batch_data, fixed_test_batch_labels = train_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        eval_ops_results = sess.run(
            eval_ops, 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        loss_val, accu_val = eval_ops_results[0:2]
        if i % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 500 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })[0]
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

In [ ]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

In [ ]:
# 命令
# tensorboard --logdir=train:'train',test:'test'